# Painfully slow and doesn't always work

In [8]:
import random
import math
from typing import List, Optional, Tuple

# ---------- helper: compute c from zeros ----------
def compute_c_from_zeros(zeros: List[int]) -> int:
    """
    zeros must be sorted ascending, length = b
    c = sum_{j=0}^{b-1} 3^{b-1-j} * 2^{zeros[j]}
    """
    if not zeros:
        return 0
    b = len(zeros)
    return sum((3 ** (b - j - 1)) * (1 << zeros[j]) for j in range(b))

# ---------- decompose_c but constrained to indices < a ----------
def decompose_c_constrained(c: int, b: int, a: int, max_z: int = None) -> Optional[List[int]]:
    """
    Try to find z_0 < z_1 < ... < z_{b-1} with z_j in [0, a-1] such that
      c = sum_{j=0}^{b-1} 3^{b-1-j} * 2^{z_j}
    Returns the list of z_j if found, else None.
    This is a greedy/backtracking search from high digit downwards,
    respecting 0 <= z <= a-1.
    """
    if c < 0 or b < 0:
        return None
    if b == 0:
        return 0 if c == 0 else None
    max_z = a - 1 if max_z is None else min(max_z, a - 1)

    # we'll try a depth-first search with small branching: for each step consider powers-of-two <= r,
    # but only those with residue matching r mod 3.
    r = c
    chosen = [None] * b  # will fill from b-1 down to 0

    # we will perform a backtracking search with pruning
    # precompute powers of two up to max_z
    pow2 = [1 << z for z in range(max_z + 1)]

    def backtrack(j: int, r_local: int, last_z_allowed: int) -> bool:
        # j: current index (we fill chosen[j] where j goes from b-1 down to 0)
        if j < 0:
            return r_local == 0
        # residue must match a power of two modulo 3
        residue = r_local % 3
        # try candidate powers of two t = 2^z with t <= r_local and z <= last_z_allowed
        # we prefer smaller z (keeps zeros evenly distributed)
        for z in range(0, last_z_allowed + 1):
            t = pow2[z]
            if t > r_local:
                break
            if t % 3 != residue:
                continue
            if (r_local - t) % 3 != 0:
                continue
            # choose z
            chosen[j] = z
            next_r = (r_local - t) // 3
            # next z must be strictly greater than current because chosen indexes increase left->right
            # but since we're filling from j=b-1 downwards, the next call should have last_z_allowed >= z+1
            # so we enforce that later.
            if backtrack(j - 1, next_r, min(max_z, z - 1)):  # ensure strict increasing after reverse
                return True
            # else try next z
        # no candidate worked
        return False

    # Because we fill from high-order term downwards, we need the chosen sequence to be strictly increasing
    # when reversed. The backtrack above enforces a decreasing sequence of z's as we go j=b-1..0,
    # so we must allow appropriate bounds. To simplify, we impose last_z_allowed = max_z initially,
    # but the recursive step enforces z decreasing; when reversed the sequence will be increasing.
    ok = backtrack(b - 1, r, max_z)
    if not ok:
        return None
    z_list = list(reversed(chosen))
    # final verify and bounds
    if any(z is None or z < 0 or z >= a for z in z_list):
        return None
    if any(z_list[i] >= z_list[i + 1] for i in range(len(z_list) - 1)):
        return None
    # verify numerically
    if compute_c_from_zeros(z_list) != c:
        return None
    return z_list

# ---------- local-search routine ----------
def find_binary_by_local_search(a: int, b: int, c_target: int,
                                force_last1: int = 3,
                                iters: int = 2000000,
                                restarts: int = 8,
                                seed: Optional[int] = None) -> Optional[str]:
    """
    Try to find an ordered zero-index list of length b (zeros ∈ [0,a-1-force_last1])
    such that compute_c_from_zeros(zeros) == c_target.
    Returns the binary string when found, otherwise None.
    force_last1: number of trailing bits forced to '1' (so zeros cannot be in last force_last1 positions).
    """

    if seed is not None:
        random.seed(seed)

    if b == 0:
        # trivial
        if c_target == 0:
            return "1" * a
        return None

    max_pos = a - 1 - (force_last1 - 1)
    if max_pos < 0:
        return None

    # build initial even spacing function
    def even_initial():
        # Spread b zeros across positions [0, max_pos] roughly evenly
        if b == 1:
            return [max_pos // 2]
        step = (max_pos + 1) / (b + 1)
        zeros = []
        for j in range(b):
            pos = int(round((j + 1) * step)) - 1
            pos = max(0, min(max_pos, pos))
            zeros.append(pos)
        # ensure strictly increasing
        for i in range(1, len(zeros)):
            if zeros[i] <= zeros[i - 1]:
                zeros[i] = min(max_pos, zeros[i - 1] + 1)
        # if last exceeds, shift left
        for i in reversed(range(b)):
            if i > 0 and zeros[i] <= zeros[i-1]:
                zeros[i-1] = max(0, zeros[i] - 1)
        return zeros

    # objective
    def err(zeros):
        return abs(compute_c_from_zeros(zeros) - c_target)

    # single restart local search
    for restart in range(restarts):
        zeros = even_initial()
        best_zeros = zeros[:]
        best_err = err(zeros)
        # small random jitter initially for diversity
        for _ in range(random.randint(0, 5)):
            i = random.randrange(b)
            if i == 0:
                low = 0
            else:
                low = best_zeros[i - 1] + 1
            high = min(max_pos, best_zeros[i + 1] - 1) if i + 1 < b else max_pos
            if low <= high:
                best_zeros[i] = random.randint(low, high)
        zeros = best_zeros[:]
        best_err = err(zeros)
        if best_err == 0:
            # verify decomposition (ensures indices within 0..a-1)
            z_try = decompose_c_constrained(c_target, b, a)
            if z_try is not None:
                # build string and return
                bits = ["1"] * a
                for zi in z_try:
                    bits[zi] = "0"
                return "".join(bits)

        # Simulated annealing style
        T0 = max(1.0, float(best_err))
        for t in range(iters):
            T = T0 * (0.995 ** t)  # cooling
            # propose a move: choose one of several move types
            move_type = random.choices(
                ["shift", "swap", "random_reposition", "global_random"], [0.6, 0.15, 0.2, 0.05]
            )[0]

            cand = zeros[:]
            if move_type == "shift":
                # shift one zero left or right by one (if possible)
                i = random.randrange(b)
                dir = random.choice([-1, 1])
                newpos = cand[i] + dir
                low_bound = 0 if i == 0 else cand[i - 1] + 1
                high_bound = max_pos if i == b - 1 else cand[i + 1] - 1
                if low_bound <= newpos <= high_bound:
                    cand[i] = newpos
            elif move_type == "swap" and b >= 2:
                # swap two adjacent zeros (effectively move them)
                i = random.randrange(b - 1)
                # try swap if keeps ordering in bounds
                cand[i], cand[i + 1] = cand[i + 1], cand[i]
                # if equals or invalid then shuffle slightly
                if not all(0 <= cand[j] <= max_pos for j in range(b)) or any(cand[j] >= cand[j + 1] for j in range(b - 1)):
                    cand = zeros[:]
            elif move_type == "random_reposition":
                # pick an index and reposition uniformly in allowed interval
                i = random.randrange(b)
                low = 0 if i == 0 else cand[i - 1] + 1
                high = max_pos if i == b - 1 else cand[i + 1] - 1
                if low <= high:
                    cand[i] = random.randint(low, high)
            else:
                # global_random: randomize all positions (but keep order)
                cand = sorted(random.sample(range(0, max_pos + 1), b))

            # compute candidate error
            e_old = best_err
            e_new = err(cand)
            accept = False
            if e_new < e_old:
                accept = True
            else:
                # accept with Metropolis criterion
                if T > 1e-12 and random.random() < math.exp(-(e_new - e_old) / T):
                    accept = True

            if accept:
                zeros = cand
                if e_new < best_err:
                    best_err = e_new
                    best_zeros = cand[:]
                    # if exact match, attempt constrained decomposition to ensure correctness
                    if best_err == 0:
                        z_try = decompose_c_constrained(c_target, b, a)
                        if z_try is not None:
                            bits = ["1"] * a
                            for zi in z_try:
                                bits[zi] = "0"
                            return "".join(bits)
            # small early exit if we find exact by compute_c (no need to decompose)
            if best_err == 0:
                z_try = decompose_c_constrained(c_target, b, a)
                if z_try is not None:
                    bits = ["1"] * a
                    for zi in z_try:
                        bits[zi] = "0"
                    return "".join(bits)

        # end iterations of this restart; maybe try a decomposition on best_zeros directly
        if best_err == 0:
            z_try = decompose_c_constrained(c_target, b, a)
            if z_try is not None:
                bits = ["1"] * a
                for zi in z_try:
                    bits[zi] = "0"
                return "".join(bits)
        # otherwise continue with next restart (randomize initial guess a bit)
    # no solution found within budget
    return None

# ----------------- example usage -----------------
if __name__ == "__main__":
    a = 18
    b = 5
    c_target = 24976
    # force last three bits to 1 (so zeros in positions 0..14)
    result = find_binary_by_local_search(a, b, c_target, force_last1=3, iters=5000, restarts=20, seed=1)
    print("result:", result)


result: None


In [9]:
find_binary_by_local_search(18, 5, 24976)